In [ ]:
import numpy as np
import torch
from torchvision import datasets, transforms
import torch.nn as nn
import plotly.graph_objs as go
import glob
import matplotlib.pyplot as plt
torch.manual_seed(0)
torch.cuda.manual_seed(0)
np.random.seed(0)
torch.backends.cudnn.deterministic = True
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
def load_dog():
  !export KAGGLE_USERNAME="Name" && export KAGGLE_KEY="Key" && mkdir -p data_dogs && cd data_dogs && kaggle datasets download -d eward96/dog-breed-images && unzip -n dog-breed-images.zip && rm dog-breed-images.zip
  !ls data
  data_dir = 'data_dogs'
  list_imgs = glob.glob(data_dir + "/**/*.jpg")
  tc = transforms.Compose([
          transforms.Resize((256, 256)),
          transforms.ToTensor()
      ])

  image_datasets = datasets.ImageFolder(data_dir, transform=tc)
  dloader = torch.utils.data.DataLoader(image_datasets, batch_size=10, shuffle=True)
  print(f"There are {len(list_imgs)} images in the dataset {data_dir}")
  return ['dogs', dloader]

def load_stl10():
  tensor_transform = transforms.ToTensor()
  # Download the MNIST Dataset
  dtwo = datasets.STL10(root = "./data",
                          download = True,
                          train = True,
                          transform = tensor_transform) #train = True,

  # DataLoader is used to load the dataset
  # for training
  dloader = torch.utils.data.DataLoader(dataset = dtwo,
                                      batch_size = 10,
                                      shuffle = True)
  return ['STL10', dloader]

def load_cifar10():
  # Download the MNIST Dataset
  transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    transforms.Grayscale() # Linear
])
  train_data = datasets.CIFAR10(root = "./data",download = True,train = True,transform = transform) #train = True,
  test_data = datasets.CIFAR10(root = "./data",download = True,train = False,transform = transform) #train = True,

  # DataLoader is used to load the dataset
  # for training
  train_dloader = torch.utils.data.DataLoader(dataset = train_data,batch_size =64,shuffle = False)
  test_dloader = torch.utils.data.DataLoader(dataset = test_data,batch_size = 64,shuffle = False)

  return ['CIFAR10', train_dloader, test_dloader]
def load_data(datasets=[load_cifar10]):
  data_loaders = []
  for data in datasets:
    try:
      data_loaders.append(data())
    except:
      print(f'The function {str(data)} does not exist')

    return data_loaders


In [ ]:
class AE_Linear01(torch.nn.Module):
  def __init__(self):
    super().__init__()

    self.encoder = torch.nn.Sequential(
        nn.Linear(32*32, 26 * 26),
        nn.ReLU(),
        nn.Linear(26*26, 512)

    )
    self.decoder = torch.nn.Sequential(
        nn.Linear(512, 26*26),
        nn.ReLU(),
        nn.Linear(26*26, 32*32),
        nn.Tanh(),
      )
  def forward(self,x):
    encoded = self.encoder(x)
    decoded = self.decoder(encoded)
    return decoded

In [ ]:
def train(dloader):
  # Model Initialization
  model = AE_Linear01().to(DEVICE)

  # Validation using MSE Loss function
  loss_function = torch.nn.MSELoss()

  # Using an Adam Optimizer with lr = 0.1
  optimizer = torch.optim.Adam(model.parameters(),
                              lr = 1,
                              weight_decay =1e-8)
  epochs=100
  outputs = []
  losses = []
  reconstructed = 0
  for epoch in range(epochs):
    for (image,_) in dloader:
      image = image.to(DEVICE)
      image = image.reshape(-1, 32*32) # Linear
      reconstructed = model(image)
      loss = loss_function(reconstructed,image)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      losses.append(loss.item())
      break
    outputs.append((epochs, image, reconstructed))
  return losses, model

In [ ]:
def test(dloader, model):
  losses = []
  for param in model.parameters():
    param.requires_grad = False
  loss_function = torch.nn.MSELoss()
  for (image,_) in dloader:
    image = image.to(DEVICE)
    image = image.reshape(-1, 32*32)
    reconstructed = model(image)
    loss = loss_function(reconstructed,image)
    losses.append(loss.item())
  return losses

In [ ]:
def training_pipeline():
  dloader = load_data()
  train_losses, model = train(dloader[0][1])
  test_losses = test(dloader[0][2], model)
  fig=go.Figure([go.Scatter(y=train_losses),go.Scatter(y=test_losses) ])#
  fig.show()
  torch.save(model.state_dict(), '/content/model_weights.pth')

training_pipeline()

Files already downloaded and verified
Files already downloaded and verified
